In [1]:
import tensorflow as tf
from tensorflow import keras
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
from imutils import paths
import random
import cv2
from tensorflow.keras.preprocessing.image import img_to_array, load_img
from sklearn.utils import shuffle
from sklearn.metrics import accuracy_score
from tensorflow.keras.layers import Input, GlobalAveragePooling2D, Dense, Dropout, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import confusion_matrix, classification_report
from tensorflow.keras.optimizers import Adam
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
import flwr as fl
# Load and compile Keras model


In [2]:
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
  tf.config.experimental.set_memory_growth(gpu, True)


In [3]:
BATCH_SIZE = 8
train_data_dir = 'D:/TQ/Federated/3/train/'

test_data_dir = 'D:/TQ/Federated/3/val/'


In [4]:

train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2  # Set the validation split percentage
)
test_datagen = ImageDataGenerator(rescale=1./255,
                                  shear_range=0.2,
                                  zoom_range=0.2,
                                  horizontal_flip=True)




In [5]:
print("Class Counts in Training Samples:")
total_classes = sorted(os.listdir(train_data_dir))
print(total_classes)
for category in total_classes:
    category_path = os.path.join(train_data_dir, category)
    sample_count = len(os.listdir(category_path))
    print(f"{category}: {sample_count} samples")


Class Counts in Training Samples:
['Animal_Abuse', 'Arson', 'Fight', 'Normal', 'Riot', 'Traffic_Accident', 'Train_Accident']
Animal_Abuse: 40 samples
Arson: 240 samples
Fight: 880 samples
Normal: 199 samples
Riot: 89 samples
Traffic_Accident: 160 samples
Train_Accident: 136 samples


In [6]:
train_classes = sorted(os.listdir(train_data_dir))
test_classes = sorted(os.listdir(test_data_dir))


In [7]:
base_model = tf.keras.applications.MobileNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
# Freeze the base model layers
base_model.trainable = False

# Add custom classification layers on top of the base model
inputs = Input(shape=(224, 224, 3))
x = base_model(inputs, training=False)
x = GlobalAveragePooling2D()(x)
x = Dropout(0.5)(x)  # Add dropout layer with a dropout rate of 0.5
outputs = Dense(7, activation='softmax')(x)  # Assuming binary classification

# Create the full model
model3 = Model(inputs, outputs)
optimizer = Adam(learning_rate=0.001)
model3.compile(optimizer=optimizer,
              loss='categorical_crossentropy',
              metrics=['accuracy'])


C:\Users\cslen\AppData\Local\Programs\Python\Python310\lib\random.py:370: DeprecationWarning: non-integer arguments to randrange() have been deprecated since Python 3.10 and will be removed in a subsequent version
  return self.randrange(a, b+1)


In [8]:
traingen = train_datagen.flow_from_directory(train_data_dir,
                                                   target_size=(225, 225),
                                                   class_mode='categorical',
                                                   classes=total_classes,
                                                   subset='training',
                                                   batch_size=BATCH_SIZE, 
                                                   shuffle=True,
                                                   seed=42)

validgen = train_datagen.flow_from_directory(train_data_dir,
                                               target_size=(225, 225),
                                               class_mode='categorical',
                                               classes=total_classes,
                                               subset='validation',
                                               batch_size=BATCH_SIZE,
                                               shuffle=True,
                                               seed=42)

testgen = test_datagen.flow_from_directory(test_data_dir,
                                             target_size=(225, 225),
                                             class_mode='categorical',
                                             classes=total_classes,
                                             batch_size=1,
                                             shuffle=False,
                                             seed=42)


Found 1397 images belonging to 7 classes.
Found 347 images belonging to 7 classes.
Found 475 images belonging to 7 classes.


In [9]:
early_stopping = EarlyStopping(monitor="val_loss", patience=10, restore_best_weights=True)
callbacks = [early_stopping]


In [10]:
history = model3.fit(
            traingen,
            epochs=5,
            steps_per_epoch=len(traingen),
            validation_data=validgen,
            validation_steps=len(validgen),
            callbacks=callbacks,
            verbose = 1
)

C:\Users\cslen\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\preprocessing\image.py:2616: DeprecationWarning: Please import `affine_transform` from the `scipy.ndimage` namespace; the `scipy.ndimage.interpolation` namespace is deprecated and will be removed in SciPy 2.0.0.
  ndimage.interpolation.affine_transform(


Epoch 1/5
175/175 [==============================] - 44s 235ms/step - loss: 0.7614 - accuracy: 0.7602 - val_loss: 1.1199 - val_accuracy: 0.7032
Epoch 2/5
175/175 [==============================] - 34s 194ms/step - loss: 0.1744 - accuracy: 0.9556 - val_loss: 1.3120 - val_accuracy: 0.6916
Epoch 3/5
175/175 [==============================] - 35s 202ms/step - loss: 0.1101 - accuracy: 0.9771 - val_loss: 1.5218 - val_accuracy: 0.6484
Epoch 4/5
175/175 [==============================] - 35s 200ms/step - loss: 0.0860 - accuracy: 0.9792 - val_loss: 1.5485 - val_accuracy: 0.7003
Epoch 5/5
175/175 [==============================] - 36s 207ms/step - loss: 0.0741 - accuracy: 0.9807 - val_loss: 1.9630 - val_accuracy: 0.6138


In [11]:
#model = tf.keras.models.load_model("Client3_Centralized.h5")

In [12]:
y_true = testgen.classes

# Calculate predictions on the test data
y_pred = model3.predict(testgen)
y_pred_labels = np.argmax(y_pred, axis=1)  # Get the predicted class labels

475/475 [==============================] - 13s 25ms/step


In [13]:
print("Accuracy:")
test_loss, test_accuracy = model3.evaluate(testgen, steps=len(testgen), verbose=0)
print(f"Test Loss: {test_loss}")
print(f"Test Accuracy: {test_accuracy}")


Accuracy:
Test Loss: 0.3339509963989258
Test Accuracy: 0.9263157844543457


In [14]:
# Calculate the confusion matrix
cm2 = confusion_matrix(y_true, y_pred_labels)
print("Confusion Matrix Before Aggregation:")
print(cm2)

Confusion Matrix Before Aggregation:
[[ 10   0   0   0   0   0   0]
 [  0  51   1   8   0   0   0]
 [  0   0 207  13   0   0   0]
 [  0   0   2  45   0   3   0]
 [  0   0   0   7  54   0   0]
 [  0   0   0   1   0  39   0]
 [  0   0   0   3   0   0  31]]


In [27]:
from sklearn.metrics import confusion_matrix
import numpy as np


# We will store the results in a dictionary for easy access later
per_class_accuracies = {}

# Calculate the accuracy for each one of our classes
for idx, cls in enumerate(total_classes):
    # True negatives are all the samples that are not our current GT class (not the current row) 
    # and were not predicted as the current class (not the current column)
    true_negatives = np.sum(np.delete(np.delete(cm, idx, axis=0), idx, axis=1))
    
    # True positives are all the samples of our current GT class that were predicted as such
    true_positives = cm2[idx, idx]
    
    # The accuracy for the current class is the ratio between correct predictions to all predictions
    per_class_accuracies[cls] = (true_positives + true_negatives) / np.sum(cm2)
print(per_class_accuracies)

{'Animal_Abuse': 1.0, 'Arson': 0.9789473684210527, 'Fight': 0.9726315789473684, 'Normal': 0.8778947368421053, 'Riot': 0.9852631578947368, 'Traffic_Accident': 0.9852631578947368, 'Train_Accident': 0.9936842105263158}


In [31]:
from sklearn.metrics import confusion_matrix
import numpy as np


# We will store the results in a dictionary for easy access later
per_class_accuracies = {}

# Calculate the accuracy for each one of our classes
for idx, cls in enumerate(total_classes):
    true_positives = cm2[idx, idx]
    
    # Total instances of the current class
    total_instances = np.sum(cm2[idx, :])
    
    # The accuracy for the current class is the ratio between correct predictions to all predictions
    per_class_accuracies[cls] = true_positives / total_instances if total_instances > 0 else 0
    print(f"{cls} - True Positives: {true_positives}, Total Instances: {total_instances}")

print(per_class_accuracies)

Animal_Abuse - True Positives: 10, Total Instances: 10
Arson - True Positives: 51, Total Instances: 60
Fight - True Positives: 207, Total Instances: 220
Normal - True Positives: 45, Total Instances: 50
Riot - True Positives: 54, Total Instances: 61
Traffic_Accident - True Positives: 39, Total Instances: 40
Train_Accident - True Positives: 31, Total Instances: 34
{'Animal_Abuse': 1.0, 'Arson': 0.85, 'Fight': 0.9409090909090909, 'Normal': 0.9, 'Riot': 0.8852459016393442, 'Traffic_Accident': 0.975, 'Train_Accident': 0.9117647058823529}


In [15]:
import pandas as pd
from openpyxl import Workbook

categories = ['Animal_Abuse', 'Arson', 'Fight', 'Normal', 'Riot', 'Traffic_Accident', 'Train_Accident']
df_cm = pd.DataFrame(cm2, index=categories, columns=categories)

# Write the confusion matrix DataFrame to an Excel file
excel_writer = pd.ExcelWriter('D:/TQ/Federated/confusion_matrix.xlsx')
# Create an Excel workbook and write the confusion matrix DataFrame to a sheet
wb = Workbook()
ws = wb.active
ws.title = 'Confusion Matrix'

# Write the column names
ws.append([''] + df_cm.columns.tolist())

# Write the confusion matrix data
for index, row in df_cm.iterrows():
    ws.append([index] + row.tolist())

# Save the workbook to an Excel file
wb.save('D:/TQ/Federated/confusion_matrix3.xlsx')

print("Confusion matrix written to confusion_matrix.xlsx")

Confusion matrix written to confusion_matrix.xlsx


In [16]:
model3.save("Client3_Centralized.h5")

In [17]:
from sklearn.metrics import classification_report 
print(classification_report(y_true, y_pred_labels, target_names=total_classes))


                  precision    recall  f1-score   support

    Animal_Abuse       1.00      1.00      1.00        10
           Arson       1.00      0.85      0.92        60
           Fight       0.99      0.94      0.96       220
          Normal       0.58      0.90      0.71        50
            Riot       1.00      0.89      0.94        61
Traffic_Accident       0.93      0.97      0.95        40
  Train_Accident       1.00      0.91      0.95        34

        accuracy                           0.92       475
       macro avg       0.93      0.92      0.92       475
    weighted avg       0.94      0.92      0.93       475



In [23]:
client3 = tf.keras.models.load_model('client3_30_fedavg.h5')

C:\Users\cslen\AppData\Local\Programs\Python\Python310\lib\random.py:370: DeprecationWarning: non-integer arguments to randrange() have been deprecated since Python 3.10 and will be removed in a subsequent version
  return self.randrange(a, b+1)


In [24]:
y_true = testgen.classes

# Calculate predictions on the test data
y_pred = client3.predict(testgen)
y_pred_labels = np.argmax(y_pred, axis=1)  # Get the predicted class labels
test_loss, test_accuracy = client3.evaluate(testgen, steps=len(testgen), verbose=0)
print(f"Test Loss: {test_loss}")
print(f"Test Accuracy: {test_accuracy}")

print(classification_report(y_true, y_pred_labels, target_names=total_classes))


C:\Users\cslen\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\preprocessing\image.py:2616: DeprecationWarning: Please import `affine_transform` from the `scipy.ndimage` namespace; the `scipy.ndimage.interpolation` namespace is deprecated and will be removed in SciPy 2.0.0.
  ndimage.interpolation.affine_transform(


475/475 [==============================] - 9s 17ms/step
Test Loss: 0.5552966594696045
Test Accuracy: 0.88210529088974
                  precision    recall  f1-score   support

    Animal_Abuse       1.00      1.00      1.00        10
           Arson       0.98      0.87      0.92        60
           Fight       1.00      0.80      0.89       220
          Normal       0.46      0.90      0.61        50
            Riot       1.00      1.00      1.00        61
Traffic_Accident       0.87      1.00      0.93        40
  Train_Accident       1.00      0.91      0.95        34

        accuracy                           0.87       475
       macro avg       0.90      0.93      0.90       475
    weighted avg       0.93      0.87      0.89       475



In [25]:
# Calculate the confusion matrix
cm = confusion_matrix(y_true, y_pred_labels)
print("Confusion Matrix Before Aggregation:")
print(cm)

Confusion Matrix Before Aggregation:
[[ 10   0   0   0   0   0   0]
 [  0  52   0   8   0   0   0]
 [  0   1 176  42   0   1   0]
 [  0   0   0  45   0   5   0]
 [  0   0   0   0  61   0   0]
 [  0   0   0   0   0  40   0]
 [  0   0   0   3   0   0  31]]


In [26]:
import pandas as pd
from openpyxl import Workbook

categories = ['Animal_Abuse', 'Arson', 'Fight', 'Normal', 'Riot', 'Traffic_Accident', 'Train_Accident']
df_cm = pd.DataFrame(cm, index=categories, columns=categories)

# Write the confusion matrix DataFrame to an Excel file
excel_writer = pd.ExcelWriter('D:/TQ/Federated/confusion_matrix.xlsx')
# Create an Excel workbook and write the confusion matrix DataFrame to a sheet
wb = Workbook()
ws = wb.active
ws.title = 'Confusion Matrix'

# Write the column names
ws.append([''] + df_cm.columns.tolist())

# Write the confusion matrix data
for index, row in df_cm.iterrows():
    ws.append([index] + row.tolist())

# Save the workbook to an Excel file
wb.save('D:/TQ/Federated/confusion_matrix3f.xlsx')

print("Confusion matrix written to confusion_matrix.xlsx")

Confusion matrix written to confusion_matrix.xlsx


In [28]:
from sklearn.metrics import confusion_matrix
import numpy as np


# We will store the results in a dictionary for easy access later
per_class_accuracies = {}

# Calculate the accuracy for each one of our classes
for idx, cls in enumerate(total_classes):
    # True negatives are all the samples that are not our current GT class (not the current row) 
    # and were not predicted as the current class (not the current column)
    true_negatives = np.sum(np.delete(np.delete(cm, idx, axis=0), idx, axis=1))
    
    # True positives are all the samples of our current GT class that were predicted as such
    true_positives = cm[idx, idx]
    
    # The accuracy for the current class is the ratio between correct predictions to all predictions
    per_class_accuracies[cls] = (true_positives + true_negatives) / np.sum(cm)
print(per_class_accuracies)

{'Animal_Abuse': 1.0, 'Arson': 0.9810526315789474, 'Fight': 0.9073684210526316, 'Normal': 0.8778947368421053, 'Riot': 1.0, 'Traffic_Accident': 0.9873684210526316, 'Train_Accident': 0.9936842105263158}


In [30]:
from sklearn.metrics import confusion_matrix
import numpy as np


# We will store the results in a dictionary for easy access later
per_class_accuracies = {}

# Calculate the accuracy for each one of our classes
for idx, cls in enumerate(total_classes):
    true_positives = cm[idx, idx]
    
    # Total instances of the current class
    total_instances = np.sum(cm[idx, :])
    
    # The accuracy for the current class is the ratio between correct predictions to all predictions
    per_class_accuracies[cls] = true_positives / total_instances if total_instances > 0 else 0
    print(f"{cls} - True Positives: {true_positives}, Total Instances: {total_instances}")

print(per_class_accuracies)

Animal_Abuse - True Positives: 10, Total Instances: 10
Arson - True Positives: 52, Total Instances: 60
Fight - True Positives: 176, Total Instances: 220
Normal - True Positives: 45, Total Instances: 50
Riot - True Positives: 61, Total Instances: 61
Traffic_Accident - True Positives: 40, Total Instances: 40
Train_Accident - True Positives: 31, Total Instances: 34
{'Animal_Abuse': 1.0, 'Arson': 0.8666666666666667, 'Fight': 0.8, 'Normal': 0.9, 'Riot': 1.0, 'Traffic_Accident': 1.0, 'Train_Accident': 0.9117647058823529}
